In [ ]:
from display import display_figures

from include.load_data import get_counts
from include.estim import Rt_MLE as estimMLE
from include.estim import Rt_U_M as estimPL
from include.estim import Rt_Cori as estimCori
from include.estim import Rt_UO_MO as estimJ
from include.optim_tools import slidingMAD

### Data fetching

dataBasis choice between 'SPF' (Santé Publique France) and 'JHU' (Johns Hopkins University)
&nbsp;
country choice should only be
   * 'France' for 'SPF'
   * any country available at https://coronavirus.jhu.edu/data for 'JHU'

In [ ]:
# Choice of country, dates, regularization parameters & computation
dataBasis = 'JHU'
country = 'France'
firstDay = '2021-11-01'  # first day of the chosen time period
lastDay = '2022-08-03'  # last day of the chosen time period

# Fetch data and associated dates
dates, data = get_counts.get_real_counts(country, firstDay, lastDay, dataBasis)
fig, axes, formattedDates = display_figures.display_data(dates, data, "Daily new cases %s for %s, %s and %s" % (dataBasis, country, firstDay, lastDay))

### Maximum Likelihood Estimator
* $\mathsf{R}^{\mathsf{MLE}} \in \mathrm{arg\,min}_{\mathsf{R} \in \mathbb{R}^{\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z}^{\mathsf{denoised}}, \mathsf{R})$

In [ ]:
# Analytic computation
R_MLE, datesMLE, dataMLE = estimMLE.Rt_MLE(dates, data)

# Display figures
figMLE, axesMLE, formattedDatesMLE = display_figures.display_REstim(datesMLE, dataMLE, R_MLE, 'MLE', dataUnder=True)

### Cori Estimator

Following the article "A New Framework and Software to Estimate Time-Varying Reproduction Numbers During Epidemics", Cori et al., 2013, Am. Journal of Epidemiology, R estimation is made using Bayesian framework resulting in:

&nbsp;
$\mathsf{R}^{\mathsf{Cori}} = \frac{1 + \sum_{s=t-\tau}^{t}\mathsf{Z}_s}{5 + \sum_{s=t-\tau}^{t}(\Phi\star\mathsf{Z})_s}$

Here $\tau = 15$

In [ ]:
# Analytic computation
R_Cori, datesCori, dataCori = estimCori.Rt_C(dates, data)

# Display results
figCori, axesCori, formattedDatesCori = display_figures.display_REstim(datesCori, dataCori, R_Cori, 'Cori', dataUnder=True)

### Penalized Likelihood estimation

1. $\mathsf{Z}^{\mathsf{denoised}} = \mathsf{slidingMAD}(\mathsf{Z}, \alpha)$
    &nbsp;

    with sliding Median Absolute Deviation (MAD) that can be tuned using $\alpha \geq 0$:
    - small $\alpha$ : lot of data denoised
    - great $\alpha$ : few data denoised

2. Solving the Penalized Likelihood functional with Chambolle-Pock algorithm
    &nbsp;
   $\mathsf{R}^{\mathsf{PL}} \in \mathrm{arg\,min}_{\mathsf{R} \in \mathbb{R}^{\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z}^{\mathsf{denoised}}, \mathsf{R}) +\mu_\mathsf{R} \|\boldsymbol{\mathsf{D}}_2\mathsf{R}\|_1$

In [ ]:
# from include.estim import Rt_U_M as estimPL
# Choice of denoising parameter alpha and regularization parameter muR
alpha = 0.1
muR = 50

# Denoising with sliding MAD then solving with Chambolle-Pock algorithm
denoisedData = slidingMAD.slidingMAD(data, alpha)
R_PL, datesPL, denoisedDataPL = estimPL.Rt_U_M(dates, denoisedData, muR=muR)

# Display results
dataCropped = data[1:]  # data[0] is used for burn-in
figPL, axesPL, formattedDatesPL = display_figures.display_REstim(datesPL, dataCropped, R_PL, 'PL', OEstimate=dataCropped - denoisedDataPL, dataUnder=True)

### Joint estimation

Solving the following Joint penalized likelihood functional using Chambolle-Pock algorithm
&nbsp;
$\mathsf{R}^{\mathsf{Joint}}, \mathsf{O}^{\mathsf{Joint}} \in \mathrm{arg\,min}_{\mathsf{R}, \mathsf{O} \in \mathbb{R}^{\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z}, \mathsf{R}, \mathsf{O}) +\lambda_\mathsf{R} \|\boldsymbol{\mathsf{D}}_2\mathsf{R}\|_1 + \lambda_\mathsf{O} \|\mathsf{O}\|_1$

* A greater $\lambda_R$ means less slope ruptures
* A greater $\lambda_O$ means less denoising
&nbsp;

* For examples, see settings.RegularizationSettings

In [ ]:
# Choice of denoising parameter alpha and regularization parameter muR
lambdaR = 3.5
lambdaO = 0.03

# Other options
# lambdaR = 50
# lambdaO = 0.75

# Solving with Chambolle-Pock algorithm
R_Joint, O_Joint, datesJ, dataJ = estimJ.Rt_UO_MO(dates, data, lambdaR=lambdaR, lambdaO=lambdaO)

# Display results
figJoint, axesJoint, formattedDatesJoint = display_figures.display_REstim(datesJ, dataJ, R_Joint, 'Joint', OEstimate=O_Joint, dataUnder=True)

In [ ]:
# Gather all methods
methods = ['MLE', 'Cori', 'PL', 'Joint']
REstimations = {'MLE': R_MLE,
                'Cori': R_Cori,
                'PL': R_PL,
                'Joint': R_Joint}

OEstimations = {'PL': dataCropped - denoisedDataPL,
                'Joint': O_Joint}

# Display comparison between methods
figComp, axesComp, formattedDatesComp = display_figures.display_REstim(datesJ, dataJ, REstimations, methods, OEstimate=OEstimations, comparison=True, dataUnder=True)